# Aprendizaje Automático - Intermedio

## Data leakage

* [Introducción](#introduccion)
  * [Fuga objetivo](#fuga_objetivo)
  * [Contaminación entramiento-prueba](#contaminacion_train_test)
* [Ejemplo](#ejemplo)
* [Conclusión](#conclusion1)
* [Ejercicio](#ejercicio)
  * [La ciencia de datos de cordones](#laciencia)
  * [El regreso de los cordones](#elregreso)
  * [¿Hacerse rico con las criptomonedas?](#hacerserico)
  * [Prevención de infecciones](#prevencion)
  * [Conclusiones](#conclusion2)

<a name="introduccion"></a>
### Introducción

En este tutorial, aprenderemos qué es la fuga de datos y cómo prevenirla. Si no sabemos prevenirlo, las fugas aparecerán con frecuencia y arruinarán nuestros modelos de manera sutil y peligrosa. Por tanto, este es uno de los conceptos más importantes para la práctica de los científicos de datos.

La **fuga de datos** (o **fuga**) ocurre cuando los datos de entrenamiento contienen información sobre el objetivo, pero datos similares no estarán disponibles cuando el modelo se use para la predicción. Esto conduce a un alto rendimiento en el conjunto de entrenamiento (y posiblemente incluso en los datos de validación), pero el modelo funcionará mal en producción. En otras palabras, la fuga hace que un modelo parezca preciso hasta que comience a tomar decisiones con el modelo y luego el modelo se vuelve muy inexacto.

Hay dos tipos principales de fuga: fuga objetivo (**target leakage**) y contaminación entrenamiento-prueba (**train-test contamination**).

<a name="fuga_objetivo"></a>
#### Fuga objetivo

La **fuga objetivo** se produce cuando los predictores incluyen datos que no estarán disponibles en el momento en que realice las predicciones. Es importante pensar en la fuga objetivo en términos del *tiempo* o el *orden cronológico* en que los datos están disponibles, no solo si una característica ayuda a hacer buenas predicciones.

Un ejemplo será útil. Imagina que quieres predecir quién enfermará de neumonía. Las primeras filas de datos en bruto se ven así:

![tabla_leakage](./images/tabla_leakage.png)

Las personas toman medicamentos antibióticos después de contraer neumonía para recuperarse. Los datos sin procesar muestran una fuerte relación entre esas columnas, pero `took_antibiotic_medicine` se cambia con frecuencia *después* de determinar el valor de `got_pneumonia`. Esta es la fuga objetivo.

El modelo vería que cualquiera que tenga un valor de `False` para `took_atibiotic_medicine` no tenía neumonía. Dado que los datos de validación provienen de la misma fuente que los datos de entrenamiento, el patrón se repetirá en la validación y el modelo tendrá excelentes puntuaciones de validación (o validación cruzada). Pero el modelo será muy inexacto cuando se implemente posteriormente en el mundo real, porque incluso los pacientes que contraerán neumonía aún no habrán recibido antibióticos cuando necesitemos hacer predicciones sobre su salud futura.

Para evitar este tipo de fuga de datos, se debe excluir cualquier variable actualizada (o creada) después de alcanzar el valor objetivo.

![moment_prediction](./images/moment_prediction.png)

<a name="contaminacion_train_test"></a>
#### Contaminación entrenamiento-prueba

Se produce un tipo diferente de fuga cuando no se tiene cuidado de distinguir los datos de entrenamiento de los datos de validación. Recordemos que la validación pretende ser una medida de cómo funciona el modelo en datos que no se han considerado antes. Se puede corromper este proceso de manera sutil si los datos de validación afectan el comportamiento de preprocesamiento. Esto a veces se llama **contaminación entrenamiento-prueba**.

Por ejemplo, imaginemos que se ejecuta el preprocesamiento (como ajustar un *imputer* a los valores faltantes) antes de llamar a `train_test_split()`. ¿El resulta final? El modelo puede obtener buenas puntuaciones de validación, lo que nos brinda una gran confianza en él, pero tiene un bajo rendimiento cuando se implementa para tomar decisiones. Después de todo, incorporamos datos de la validación o datos de prueba en la forma en que realiza las predicciones, por lo que puede funcionar bien en esos datos en particular, incluso si no se puede generalizar a nuevos datos. Este problema se vuelve aún más sutil (y más peligroso) cuando se realiza una ingeniería de características más compleja.

Si nuestra validación se basa en una simple división *train-test-split*, excluyamos los datos de validación de cualquier tipo de *fitting*, incluida el fitting de los pasos de preprocesamiento. Esto es más fácil si se usan pipelines scikit-learn. ¡Al usar la validación cruzada, es aún más crítico que hagamos el preprocesamiento dentro del pipeline!

<a name="ejemplo"></a>
### Ejemplo

En este ejemplo, aprenderemos una forma de detectar y eliminar las fugas objetivo.

Utilizaremos un conjunto de datos sobre aplicaciones de tarjetas de crédito. La información sobre cada solicitud de tarjeta de crédito se almacena en un DataFrame `X`. La usaremos para predecir qué aplicaciones se aceptaron en una Serie `y`.

In [11]:
import pandas as pd

# Lee los datos
data = pd.read_csv('./input/aer-credit-card-data/AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no'])

# Selecciona objetivo
y = data.card

# Selecciona predictores
X = data.drop(['card'], axis=1)

print("Número de filas del dataset: {}".format(X.shape[0]))
X.head()

Número de filas del dataset: 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


Como se trata de un pequeño conjunto de datos, utilizaremos la validación cruzada para garantizar medidas precisas de la calidad del modelo.

In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Como no hay preprocesamiento, no necesitamos una tubería (¡de todos modos, se utiliza como práctica recomendada!)
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Precisión Cross-validation: %f" % cv_scores.mean())

Precisión Cross-validation: 0.981043


Con la experiencia, descubriremos que es muy raro encontrar modelos que sean precisos el 98% del tiempo. Sucede, pero es bastante raro que debamos inspeccionar los datos más de cerca para detectar fugas objetivo. 

Aquí hay un resumen de los datos, que también puede encontrar en la pestaña de datos:

+ `card`: 1 si se acepta la solicitud de tarjeta de crédito, 0 si no
+ `reports`: número de informes derogados importantes
+ `age`: edad n años más doceavos de año
+ `income`: ingreso anual (dividido entre 10,000)
+ `share`: relación entre el gasto mensual con tarjeta de crédito y el ingreso anual
+ `expenditure`: gasto mensual promedio de tarjeta de crédito
+ `owner`: 1 si posee casa, 0 si alquila
+ `selfempl`: 1 si trabaja por cuenta propia, 0 si no
+ `dependents`: 1 + número de dependientes
+ `months`: meses viviendo en la dirección actual
+ `majorcards`: número de tarjetas de crédito principales retenidas
+ `active`: número de cuentas de crédito activas

Algunas variables parecen sospechosas. Por ejemplo, ¿**`expenditure`** significa gasto en esta tarjeta o en tarjetas usadas antes de la aplicación?

En este punto, las comparaciones de datos básicos pueden ser muy útiles:

In [14]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Fracción de aquellos que no recibieron una tarjeta y no tuvieron gastos: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Fracción de aquellos que recibieron una tarjeta y no tuvieron gastos: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Fracción de aquellos que no recibieron una tarjeta y no tuvieron gastos: 1.00
Fracción de aquellos que recibieron una tarjeta y no tuvieron gastos: 0.02


Como se muestra anteriormente, todos los que no recibieron una tarjeta no tuvieron gastos, mientras que solo el 2% de los que recibieron una tarjeta no tuvieron gastos. No es sorprendente que nuestro modelo parezca tener una alta precisión. Pero esto también parece ser un caso de fuga objetivo, donde los gastos probablemente significan los *gastos en la tarjeta que solicitaron*.

Dado que **`share`** está determinada en parte por **`expenditure`**, también debe excluirse. Las variables **`active`** y **`majorcards`** son un poco menos claras, pero a partir de la descripción, suenan preocupantes. En la mayoría de las situaciones, es mejor prevenir que curar si no puede localizar a las personas que crearon los datos para obtener más información.

Ejecutaríamos un modelo sin fuga objetivo de la siguiente manera:

In [17]:
# Eliminamos los predictores defectuosos del conjunto de datos
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Evalua el modelo con los predictores defectuosos eliminados
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Precisión Cross-validation: %f" % cv_scores.mean())

Precisión Cross-validation: 0.836227


Esta precisión es bastante menor, lo que puede ser decepcionante. Sin embargo, podemos esperar que sea correcto aproximadamente el 80% de las veces cuando se usa en nuevas aplicaciones, mientras que el modelo con fugas probablemente sería mucho peor que eso (a pesar de su puntuación aparente más alta en validación cruzada).

<a name="conclusion1"></a>
### Conclusión

La fuga de datos puede ser un error multimillonario en muchas aplicaciones de ciencia de datos. La separación cuidadosa de los datos de entrenamiento y validación puede prevenir la contaminación entrenamiento-prueba y las pipelines pueden ayudar a implementar esta separación. Del mismo modo, una combinación de precaución, sentido común y exploración de datos puede ayudar a identificar la fuga objetivo.

<a name="ejercicio"></a>
## Ejercicio

La mayoría de las personas encuentran que la fuga objetivo es muy complicada hasta que lo han pensado durante mucho tiempo. Por tanto, antes de tratar de pensar en fugas en el ejemplo del precio de la vivienda, veremos algunos ejemplos en otras aplicaciones. Las cosas se sentirán más familiares una vez que volcamos a una pregunta sobre los precios de la vivienda.

<a name="laciencia"></a>
### 1. La ciencia de datos de cordones

Nike lo contrató como consultor de ciencia de datos para ayudarlos a ahorrar dinero en materiales para zapatos. Tu primera tarea es revisar un modelo construido por uno de sus empleados para predecir cuántos cordones necesitarán cada mes. Las características del modelo de aprendizaje automático incluyen:

- El mes actual (enero, febrero, etc.)
- Gastos publicitarios en el mes anterior
- Varias características macroeconómicas (como la tasa de desempleo) a principios del mes actual
- La cantidad de cuero que terminaron usando en el mes actual

Los resultados muestran que el modelo es casi perfectamente preciso si incluye la característica sobre la cantidad de cuero que usaron. Pero solo es moderadamente preciso si deja de lado esa característica. Te das cuenta de que esto se debe a que la cantidad de cuero que usan es un indicador perfecto de cuántos zapatos producen, lo que a su vez te dice cuántos cordones de zapatos necesitan.

¿Crees que la característica _leather used_ constituye una fuente de fuga de datos? Si su respuesta es "depende", ¿de qué depende?

**Solución**: Esto es complicado y depende de los detalles de cómo se recopilan los datos (que es común cuando se piensa en fugas). ¿A principios de mes decidirías cuánto cuero se usará ese mes? Si es así, esto está bien. Pero si eso se determina durante el mes, no tendrá acceso a él cuando haga la predicción. Si tienes una conjetura al comienzo del mes y posteriormente se cambia durante el mes, la cantidad real utilizada durante el mes no se puede usar como una característica (porque causa una fuga).

<a name="elregreso"></a>
### 2. El regreso de los cordones

Tienes una nueva idea. Podrías usar la cantidad de cuero que Nike ordenó (en lugar de la cantidad que realmente usaron) antes de un mes dado como predictor en tu modelo de cordón.

¿Cambia esto su respuesta sobre si hay un problema de fuga? Si responde "depende", ¿de qué depende?

**Solución**: Esto podría estar bien, pero depende de si ordenan primero los cordones de los zapatos o el cuero primero. Si piden cordones de zapatos primero, no sabrá cuánto cuero han pedido cuando prediga sus necesidades de cordones de zapatos. Si piden cuero primero, entonces tendrá ese número disponible cuando haga su pedido de cordones de los zapatos y debería estar bien.

<a name="hacerserico"></a>
### 3. ¿Hacerse rico con las criptomonedas?

Le ahorraste a Nike tanto dinero que te dieron un bono. Felicidades.

Tu amigo, que también es científico de datos, dice que ha creado un modelo que te permitirá convertir su bono en millones de dólares. Específicamente, su modelo predice el precio de una nueva criptomoneda (como Bitcoin, pero una nueva) un día antes del momento de la predicción. Su plan es comprar la criptomoneda siempre que el modelo diga que el precio de la moneda (en dólares) está a punto de subir.

Las características más importantes en su modelo son:

- Precio actual de la moneda
- Cantidad de la moneda vendida en las últimas 24 horas.
- Cambio en el precio de la moneda en las últimas 24 horas.
- Cambio en el precio de la moneda en la última 1 hora
- Número de nuevos tweets en las últimas 24 horas que mencionan la moneda

El valor de la criptomoneda en dólares ha fluctuado hacia arriba y hacia abajo en más de 100 € en el último año y, sin embargo, el error promedio de su modelo es inferior a 1 €. Él dice que esto es una prueba de que su modelo es exacto y que debe invertir con él, comprando la moneda cada vez que el modelo dice que está a punto de subir.

¿Tiene razón? Si hay un problema con su modelo, ¿qué es?

**Solución**: no hay ninguna fuente de fuga aquí. Estas funciones deberían estar disponibles en el momento en que desee realizar una predición y es poco probable que se modifiquen en los datos de entrenamiento después de determinar el objetivo de predicción. Pero la forma en que describe la precisión podría ser engañosa si no tienes cuidado. Si el precio se mueve gradualmente, el precio de hoy será un predictor preciso del precio de mañana, pero puede no decirle si es un buen momento para invertir. Por ejemplo, si es 100 hoy, predecir un precio de 100 mañana puede parecer exacto, incluso si no puede decirle si el precio está subiendo o bajando del precio actual. Un mejor objetivo de predicción sería el cambio de precio durante el día siguiente. Si puede predecir constantemente si el precio está por subir o bajar (y por cuánto), puede tener una oportunidad de inversión ganadora.

<a name="prevencion"></a>
### 4. Prevención de infecciones

Una agencia que brinda atención médica quiere predecir qué pacientes de una cirugía rara están en riesgo de infección, por lo que puede alertar a las enfermeras para que sean especialmente cuidadosas al hacer un seguimiento de esos pacientes.

Quieres construir un modelo. Cada fila en el conjunto de datos de modelado será un solo paciente que recibió la cirugía y el objetivo de predicción será si tuvieron una infección.

Algunos cirujanos pueden realizar el procedimiento de manera que aumente o disminuya el riesgo de infección. Pero, ¿cómo puede incorporar mejor la información del cirujano en el modelo?

Tienes una idea inteligente.

1. Toma todas las cirugías de cada cirujano y calcula la tasa de infección entre esos cirujanos.
2. Para cada paciente en los datos, averigüa quién era el cirujano e incluye la tasa de infección promedio de ese cirujano como característica.

¿Esto plantea algún problema de fuga objetivo?

¿Presenta algún problema de contaminación en las pruebas de tren?

**Solución**: Esto plantea un riesgo de fuga objetivo y contaminación entrenamiento-prueba (aunque puede evitar ambos si tiene cuidado).

Tiene una fuga objetivo si el resultado de un paciente determinado contribuye a la tasa de infección de su cirujano, que luego se incluye nuevamente al modelo de predicción para determinar si ese paciente se infecta. Puedes evitar la fuga objetivo si calculas la tasa de infección del cirujano utilizando solo las cirugías antes del paciente para el que estamos prediciendo. Calcular esto para cada cirugía en tus datos de entrenamiento puede ser un poco complicado.

También tienes un problema de contaminación entrenamiento-prueba si calculas esto utilizando todas las cirugías que realizó un cirujano, incluidas las del conjunto de prueba. El resultado sería que tu modelo podría verse muy preciso en el conjunto de prueba, incluso si no se generaliza bien a los nuevos pacientes después de que se implemente el modelo. Esto sucedería porque la función de riesgo del cirujano tiene en cuenta los datos del conjunto de pruebas. Existen conjuntos de pruebas para estimar cómo funcionará el modelo al ver nuevos datos. Entonces esta contaminación anula el propósito del conjunto de prueba.

<a name="precios"></a>
### 5. Precios de la vivienda

Construirás un modelo para predecir los precios de la vivienda. El modelo se implementará de manera continua para predecir el precio de una nueva casa cuando se agrega una descripción a un sitio web. Aquí hay cuatro características que podrían usarse como predictores.

1. Tamaño de la casa (en metros cuadrados)
2. Precio promedio de venta de viviendas en el mismo barrio
3. Latitud y longitud de la casa.
4. Si la casa tiene un sótano

Tienes datos históricos para entrenar y validar el modelo.

¿Cuál de las características es más probable que sea una fuente de fuga?

**Solución**: La opción 2 es la fuente de fuga objetivo. Aquí hay un análisis para cada característica:

1. Es poco probable que el tamaño de una casa cambie después de su venta (aunque técnicamente es posible). Pero, por lo general, esto estará disponible cuando necesitemos hacer una predicción y los datos no se modificarán después de que se venda la casa. Entonces es bastante seguro.

2. No conocemos las reglas para cuando esto se actualiza. Si el campo se actualiza en los datos sin procesar después de que se vendió una casa y la venta de la casa se usa para calcular el promedio, esto constituye un caso de fuga objetivo. En el extremo, si solo se vende una casa en el vecindario y es la casa que estamos tratando de predecir, entonces el promedio será exactamente igual al valor que estamos tratando de predecir. En general, para vecindarios con pocas ventas, el modelo funcionará muy bien en los datos de capacitación. Pero cuando apliques el modelo, la casa que predices aún no se habrá vendido, por lo que esta función no funcionará de la misma manera que en los datos de capacitación.

3. Estos no cambian y estarán disponibles en el momento en que queramos hacer una predicción. Por lo tanto, no hay riesgo de fuga objetivo aquí.

4. Esto tampoco cambia y está disponible en el momento en que queremos hacer una predicción. Por lo tanto, no hay riesgo de fuga objetivo aquí.

<a name="conclusion2"></a>
### Conclusión

La fuga es un problema difícil y sutil. Deberías estar orgulloso si se dio cuenta de los problemas en estos ejemplos. Ahora tienes las herramientas para hacer modelos muy precisos y retomar los problemas prácticos más difíciles que surgen al aplicar estos modelos para resolver problemas reales.